# La celda de abajo agregarla al final de la imputación de Julio en las superficies. 

In [41]:
# Imputo los valores de superficie cubierta y total, considerando que si tengo uno solo de los dos, son el mismo valor

# Imputo superficie total con los datos de cuubierta
mask_sup_cov = (df2['surface_covered_in_m2'].notna()) & (df2['surface_total_in_m2'].isna())
print('Datos a imputar en sup_total: ',mask_sup_cov.sum())

def compl_m2_cub (row):
    if row['surface_total_in_m2']>=0:
        result = row['surface_total_in_m2']
    else:
        result = row['surface_covered_in_m2']
    return result

df2['surface_total_in_m2'] = df2.apply(compl_m2_cub, axis = 1)
display(df2.info())

# Imputo superficie cubierta con los datos de total
mask_sup_cov = (df2['surface_total_in_m2'].notna()) & (df2['surface_covered_in_m2'].isna())
print('Datos a imputar en sup_cubierta: ',mask_sup_cov.sum())

def compl_m2_tot (row):
    if row['surface_covered_in_m2']>=0:
        result = row['surface_covered_in_m2']
    else:
        result = row['surface_total_in_m2']
    return result

df2['surface_covered_in_m2'] = df2.apply(compl_m2_tot, axis = 1)
display(df2.info())

Datos a imputar en sup_total:  4887
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30952 entries, 0 to 121219
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   property_type               30952 non-null  object 
 1   place_name                  30952 non-null  object 
 2   price_aprox_local_currency  27828 non-null  float64
 3   price_aprox_usd             27828 non-null  float64
 4   surface_total_in_m2         30259 non-null  float64
 5   surface_covered_in_m2       28882 non-null  float64
 6   price_usd_per_m2            22519 non-null  float64
 7   price_per_m2                26276 non-null  float64
 8   properati_url               30952 non-null  object 
 9   description                 30952 non-null  object 
 10  title                       30952 non-null  object 
 11  image_thumbnail             30477 non-null  object 
 12  tiene_jardin                30952 non-null  float64

None

Datos a imputar en sup_cubierta:  1377
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30952 entries, 0 to 121219
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   property_type               30952 non-null  object 
 1   place_name                  30952 non-null  object 
 2   price_aprox_local_currency  27828 non-null  float64
 3   price_aprox_usd             27828 non-null  float64
 4   surface_total_in_m2         30259 non-null  float64
 5   surface_covered_in_m2       30259 non-null  float64
 6   price_usd_per_m2            22519 non-null  float64
 7   price_per_m2                26276 non-null  float64
 8   properati_url               30952 non-null  object 
 9   description                 30952 non-null  object 
 10  title                       30952 non-null  object 
 11  image_thumbnail             30477 non-null  object 
 12  tiene_jardin                30952 non-null  floa

None

# Esto se agregar en la parte de test modelado básico, después de la importación y la reducción de lugares con pocos datos, pero antes de los demás modelados, es un modelado separado sobre menos features

### Modelado SOLO con place_name y property_type y sin stratify

In [82]:
# Selecciono las columnas que considero features para modelar 
X = df2.loc[:,['property_type','place_name']]
y = df2['price_usd_per_m2']

# Convierto a dummies las categóricas
X = pd.get_dummies(X,drop_first=True)

# Separo en train/test
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=33)

X.shape, y.shape, X_train.shape,X_test.shape, y_train.shape, y_test.shape

((24041, 31), (24041,), (18030, 31), (6011, 31), (18030,), (6011,))

In [83]:
#prueba con crossvalidation con Ridge
rcv = RidgeCV(alphas=(0.0001,0.001,0.01,0.1,1,10,100,1000,10000), cv=5)

rcv.fit(X_train,y_train)
print('intercept: ',rcv.intercept_,'coef: ', rcv.coef_,'alpha: ', rcv.alpha_)

y_pred_rcv = rcv.predict(X_test)

metrics.r2_score(y_test, y_pred_rcv)

intercept:  1407.153115291106 coef:  [ 1224.48695568    21.2178587  -1001.51514817  -622.96873516
   143.5921864    160.52842616  7812.44619962  -444.12004447
  -438.52358608  -389.30925439  -564.05087969  -714.19593336
  -999.67948195  -638.19708162  -604.39579306 -1187.00404765
    75.22120581   199.43264399   359.22542444   500.48172768
  -647.2190963   2587.71815855   235.08821982  -235.58776892
  5644.6010124   -529.05364222  -282.3774712   -334.70118548
  -399.18280479  -409.85047452  -561.65550654] alpha:  10.0


0.22245471068939204